# SiFP Function Point Estimation for Target Requirements
**Automated Software Function Point (SiFP) estimation using LLM analysis of target requirements with UGEP/UGDG identification and comprehensive metrics calculation.**

In [ ]:
# Cell [0] - Setup and Imports
# Purpose: Import all required libraries and configure environment settings for SiFP Function Point Estimation
# Dependencies: os, sys, logging, asyncio, dotenv, pathlib, datetime, redis, json, src modules
# Breadcrumbs: Setup -> Imports -> Environment Configuration -> SiFP Workflow Setup

import os
import sys
import logging
import asyncio
import json
import pathlib
from datetime import datetime
from pathlib import Path
from dotenv import load_dotenv
from typing import Dict, List
from redis.asyncio import Redis
from redisvl.index import AsyncSearchIndex

def setup_environment():
    """
    Configure Python path, logging, and load environment variables for SiFP estimation
    
    Returns:
        dict: Configuration parameters including model settings and SiFP-specific settings
    """
    # Get the absolute path to the project root directory (parent of notebooks)
    notebook_dir = Path(os.getcwd())
    project_root = notebook_dir.parent
    src_path = project_root / 'src'
    
    # Add the project root to Python path if not already there
    if str(project_root) not in sys.path:
        sys.path.append(str(project_root))
    
    # Load environment variables
    load_dotenv()
    
    # Get model configuration
    current_model = os.getenv('CURRENT_MODEL', 'CLAUDE_3_5_MODEL_ID')
    model_name = os.getenv(current_model, 'claude-2.1')
    model_dir_name = model_name.replace('/', '_').replace('-', '_').lower()
    
    # Create logs directory structure
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    log_dir = project_root / 'logs' / model_dir_name
    log_dir.mkdir(parents=True, exist_ok=True)
    log_file = log_dir / f"SIFP_{model_dir_name}_{timestamp}.log"
    
    # Get SiFP-specific configuration
    requirement_type = os.getenv('SIFP_ESTIMATION_REQUIREMENT', 'TARGET').lower()
    if requirement_type not in ['source', 'target']:
        requirement_type = 'target'
    
    # Configuration from environment variables
    config = {
        'PROJECT_ROOT': project_root,
        'SRC_PATH': src_path,
        'LOG_FILE': log_file,
        'MODEL_NAME': model_name,
        'MODEL_DIR_NAME': model_dir_name,
        'CURRENT_MODEL': current_model,
        'REQUIREMENT_TYPE': requirement_type,
        'TEST_MODE': os.getenv('TEST_MODE', 'False').lower() == 'true',
        'LOG_LEVEL': os.getenv('LOG_LEVEL', 'DEBUG'),
        'NEO4J_URI': os.getenv('NEO4J_URI'),
        'NEO4J_USER': os.getenv('NEO4J_USER'),
        'NEO4J_PASSWORD': os.getenv('NEO4J_PASSWORD'),
        'NEO4J_DATABASE': os.getenv('NEO4J_DATABASE', 'neo4j'),
        'NEO4J_PROJECT_NAME': os.getenv('NEO4J_PROJECT_NAME', 'eANCI'),
        'REDIS_HOST': os.getenv('REDIS_HOST', 'localhost'),
        'REDIS_PORT': os.getenv('REDIS_PORT', 6379),
        'REDIS_PASSWORD': os.getenv('REDIS_PASSWORD', '')
    }
    
    print(f"Project root added to path: {project_root}")
    print(f"SiFP Configuration:")
    print(f"  Model: {config['MODEL_NAME']}")
    print(f"  Requirement Type: {config['REQUIREMENT_TYPE'].upper()}")
    print(f"  Test Mode: {config['TEST_MODE']}")
    print(f"  Project: {config['NEO4J_PROJECT_NAME']}")
    print(f"  Log File: {config['LOG_FILE']}")
    
    return config

# Execute setup when imported
CONFIG = setup_environment()

# Import project modules after path setup
from praxis_requirements_analyzer.utils.logger import setup_logger
from praxis_requirements_analyzer.neo4j.neo4j_client import Neo4jClient
from praxis_requirements_analyzer.redis.redis_client import RedisClient
from praxis_requirements_analyzer.llm.manager import LLMManager
from praxis_requirements_analyzer.requirements_analyzer.sifp_workflow import SIFPWorkflow
from praxis_requirements_analyzer.requirements_analyzer.sifp_prompt_manager import SIFPPromptManager
from praxis_requirements_analyzer.llm.models.huggingface.hf_embeddings_client import HuggingFaceEmbeddingsClient
from praxis_requirements_analyzer.neo4j.requirements_client import RequirementsClient
from praxis_requirements_analyzer.models.requirement import Requirement

print("Setup completed successfully!")


In [ ]:
# Cell [1] - Configure Logging System for SiFP Estimation
# Purpose: Set up comprehensive logging with file and console handlers for SiFP workflow
# Dependencies: setup_logger from Cell 0, CONFIG from Cell 0
# Breadcrumbs: Setup -> Logging Configuration -> SiFP-specific Logging

def setup_sifp_logging():
    """
    Configure comprehensive logging system for SiFP estimation workflow
    
    Returns:
        logging.Logger: Main SiFP logger instance
    """
    print(f"Configuring SiFP Logging System")
    print("=" * 80)
    print(f"Log Level: {CONFIG['LOG_LEVEL']}")
    print(f"Model: {CONFIG['MODEL_NAME']}")
    print(f"Requirement Type: {CONFIG['REQUIREMENT_TYPE'].upper()}")
    print(f"Log File: {CONFIG['LOG_FILE']}")
    
    # Configure root logger
    logging.basicConfig(
        level=getattr(logging, CONFIG['LOG_LEVEL'].upper()),
        format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
        handlers=[
            logging.FileHandler(CONFIG['LOG_FILE']),
            logging.StreamHandler()  # Also keep console output
        ]
    )

    # Set Redis logger
    redis_logger = logging.getLogger("redis")
    redis_logger.setLevel(logging.DEBUG)

    # Set all relevant SiFP loggers
    sifp_loggers = {
        "src.requirements_analyzer.sifp_workflow": logging.DEBUG,
        "src.requirements_analyzer.sifp_prompt_manager": logging.DEBUG,
        "src.llm.models.huggingface.hf_embeddings_client": logging.DEBUG,
        "src.llm.manager": logging.DEBUG,
        "src.llm.manager.llm_manager": logging.DEBUG,
        "src.llm.models.anthropic.claude_client": logging.DEBUG,
        "src.llm.models.openai.openai_client": logging.DEBUG,
        "src.llm.models.huggingface.hf_client": logging.DEBUG,
        "src.redis.redis_client": logging.DEBUG,
        "redisvl": logging.DEBUG,
        "notebooks.sifp_estimation": logging.DEBUG
    }

    # Configure all loggers to use both file and console output
    for logger_name, level in sifp_loggers.items():
        logger = logging.getLogger(logger_name)
        logger.setLevel(level)
        logger.propagate = True

    # Create main logger for this notebook
    main_logger = logging.getLogger("src.requirements_analyzer.sifp_workflow")
    main_logger.setLevel(logging.DEBUG)

    # Test logging
    main_logger.debug(f"SiFP logging initialized - writing to {CONFIG['LOG_FILE']}")
    main_logger.info(f"Using model: {CONFIG['MODEL_NAME']}")
    main_logger.info(f"Processing {CONFIG['REQUIREMENT_TYPE'].upper()} requirements")
    main_logger.info(f"Project: {CONFIG['NEO4J_PROJECT_NAME']}")
    
    if CONFIG['TEST_MODE']:
        main_logger.info(f"Running in TEST MODE")
    
    # Test all log levels
    main_logger.debug("Debug logging enabled")
    main_logger.info("Info logging test")
    main_logger.warning("Warning logging test")
    main_logger.error("Error logging test")
    
    print("SiFP logging system configured successfully!")
    return main_logger

# Execute logging setup
sifp_logger = setup_sifp_logging()


In [ ]:
# Cell [2] - Initialize Clients and SiFP Workflow
# Purpose: Initialize Neo4j, Redis, LLM components and SiFP Workflow for function point estimation
# Dependencies: Client classes from Cell 0, logging configuration from Cell 1
# Breadcrumbs: Setup -> Logging -> Client Initialization -> SiFP Workflow Setup

async def initialize_sifp_clients():
    """
    Initialize all clients and workflow components for SiFP estimation
    
    Returns:
        tuple: (neo4j_client, redis_client, workflow, requirements_client)
    """
    init_logger = logging.getLogger("sifp_initialization")
    
    try:
        init_logger.info("Starting SiFP client initialization")
        print(f"\nInitializing SiFP Clients and Workflow")
        print("=" * 80)
        
        # Initialize Neo4j client with configuration
        init_logger.debug("Initializing Neo4j client")
        print(f"Connecting to Neo4j database...")
        neo4j_client = Neo4jClient(
            uri=CONFIG['NEO4J_URI'],
            user=CONFIG['NEO4J_USER'],
            password=CONFIG['NEO4J_PASSWORD'],
            database=CONFIG['NEO4J_DATABASE']
        )
        await neo4j_client.connect()
        init_logger.info("Neo4j client connected successfully")
        print(f"Neo4j connected to database: {CONFIG['NEO4J_DATABASE']}")

        # Initialize Redis components
        init_logger.debug("Initializing Redis client")
        print(f"Setting up Redis connection...")
        
        # Build Redis URL and set environment variable for redisvl
        if CONFIG['REDIS_PASSWORD']:
            redis_url = f"redis://:{CONFIG['REDIS_PASSWORD']}@{CONFIG['REDIS_HOST']}:{CONFIG['REDIS_PORT']}"
        else:
            redis_url = f"redis://{CONFIG['REDIS_HOST']}:{CONFIG['REDIS_PORT']}"
            
        # Set REDIS_URL explicitly for redisvl library
        os.environ['REDIS_URL'] = redis_url
        
        # Create Redis base client
        redis_base = Redis.from_url(redis_url)
        
        # Verify Redis connection
        try:
            await redis_base.ping()
            init_logger.info("Redis connection verified")
            print(f"Redis connected at {CONFIG['REDIS_HOST']}:{CONFIG['REDIS_PORT']}")
        except Exception as e:
            init_logger.error(f"Redis connection failed: {str(e)}")
            print(f"Redis connection failed: {str(e)}")
            raise
        
        # Initialize our Redis client wrapper
        redis_client = RedisClient(redis_base)
        
        # Initialize indices in Redis client
        try:
            print(f"Initializing Redis indices...")
            await redis_client.initialize_indices()
            init_logger.info("Successfully created/updated Redis indices")
            print(f"Redis indices initialized successfully")
        except Exception as e:
            init_logger.error(f"Failed to initialize Redis indices: {str(e)}")
            print(f"Failed to initialize Redis indices: {str(e)}")
            raise

        # Initialize LLM components
        init_logger.debug("Initializing LLM components")
        print(f"Setting up LLM Manager...")
        llm_manager = LLMManager()
        
        # Initialize the models
        try:
            await llm_manager.initialize_models()
            init_logger.info(f"LLM manager initialized with model: {CONFIG['MODEL_NAME']}")
            print(f"LLM Manager initialized with model: {CONFIG['MODEL_NAME']}")
        except Exception as e:
            init_logger.error(f"Failed to initialize LLM models: {str(e)}")
            print(f"Failed to initialize LLM models: {str(e)}")
            raise

        # Initialize SiFP-specific components
        init_logger.debug("Initializing SiFP-specific components")
        print(f"Initializing SiFP components...")
        prompt_manager = SIFPPromptManager()
        embedding_client = HuggingFaceEmbeddingsClient()

        # Initialize requirements client with project name
        init_logger.debug("Initializing requirements client")
        requirements_client = RequirementsClient(
            neo4j_client=neo4j_client,
            project_name=CONFIG['NEO4J_PROJECT_NAME']
        )
        print(f"Requirements client initialized for project: {CONFIG['NEO4J_PROJECT_NAME']}")

        # Initialize SiFP workflow with proper components
        init_logger.debug("Initializing SiFP workflow")
        print(f"Setting up SiFP Workflow...")
        workflow = SIFPWorkflow(
            llm_manager=llm_manager,
            prompt_manager=prompt_manager,
            model_name=CONFIG['MODEL_NAME'],
            redis_client=redis_client,
            embedding_client=embedding_client
        )
        
        # Explicitly set workflow logger level
        workflow.logger.setLevel(logging.DEBUG)
        
        # Initialize RedisVL indices
        try:
            print(f"Initializing RedisVL indices...")
            await workflow.init_indices()
            init_logger.info("Successfully created RedisVL indices")
            print(f"RedisVL indices created successfully")
        except Exception as e:
            init_logger.error(f"Failed to create RedisVL indices: {str(e)}")
            # Log the Redis client state for debugging
            init_logger.debug(f"Redis client state: {redis_client.client}")
            try:
                ping_result = await redis_client.client.ping()
                init_logger.debug(f"Redis client ping: {ping_result}")
            except Exception as ping_error:
                init_logger.debug(f"Redis ping failed: {ping_error}")
            print(f"Failed to create RedisVL indices: {str(e)}")
            raise
        
        init_logger.info(f"SiFP initialization complete - all components ready")
        print(f"\nSiFP Components Initialized Successfully!")
        print("=" * 80)
        print(f"Model: {CONFIG['MODEL_NAME']}")
        print(f"Project: {CONFIG['NEO4J_PROJECT_NAME']}")
        print(f"Requirement Type: {CONFIG['REQUIREMENT_TYPE'].upper()}")
        print(f"Test Mode: {CONFIG['TEST_MODE']}")
        
        return neo4j_client, redis_client, workflow, requirements_client
        
    except Exception as e:
        init_logger.critical(f"Critical error during SiFP initialization: {str(e)}", exc_info=True)
        print(f"Critical SiFP initialization error: {str(e)}")
        raise

# Execute SiFP client initialization
init_logger = logging.getLogger("sifp_initialization")
init_logger.info("Starting SiFP client initialization process")

try:
    neo4j_client, redis_client, workflow, requirements_client = await initialize_sifp_clients()
    init_logger.info("SiFP client initialization completed successfully")
except Exception as e:
    init_logger.critical("Failed to initialize SiFP clients", exc_info=True)
    raise

In [ ]:
# Cell [3] - Define SiFP Processing Helper Functions
# Purpose: Define the main SiFP requirements processing function with detailed logging
# Dependencies: Workflow and clients from Cell 2, CONFIG from Cell 0
# Breadcrumbs: Client Initialization -> Helper Functions -> SiFP Processing Logic

async def process_sifp_requirements():
    """
    Process requirements using the initialized SiFP workflow.
    Handles fetching requirements from Neo4j and processing them through SiFP estimation.
    
    Returns:
        List[Dict]: List of processed SiFP requirement estimations
    """
    sifp_logger = setup_logger("src.requirements_analyzer.sifp_workflow", logging.DEBUG)
    
    try:
        print(f"\nStarting SiFP Requirements Processing")
        print("=" * 80)
        
        # Add detailed debug logging
        sifp_logger.debug("Starting SiFP requirements processing")
        
        # Fetch requirements from Neo4j
        print(f"Fetching requirements from Neo4j...")
        requirements = await requirements_client.get_requirements()
        sifp_logger.debug(f"Got requirements dictionary with keys: {requirements.keys()}")
        
        # Get requirement type from configuration
        requirement_type = CONFIG['REQUIREMENT_TYPE']
        sifp_logger.debug(f"Using requirement type: {requirement_type}")
        
        # Get requirements based on type
        selected_requirements = requirements[requirement_type]
        sifp_logger.debug(f"Found {len(selected_requirements)} {requirement_type} requirements")
        
        print(f"Processing Configuration:")
        print(f"   Requirement Type: {requirement_type.upper()}")
        print(f"   Total Requirements: {len(selected_requirements)}")
        print(f"   Model: {CONFIG['MODEL_NAME']}")
        print(f"   Project: {CONFIG['NEO4J_PROJECT_NAME']}")
        
        # Log each requirement being processed
        for req in selected_requirements:
            sifp_logger.debug(f"Processing requirement: ID={req.id}, Type={requirement_type.upper()}, Content length={len(req.content)}")
        
        # Apply test mode limitations if enabled
        if CONFIG['TEST_MODE']:
            original_count = len(selected_requirements)
            selected_requirements = selected_requirements[:2]
            sifp_logger.debug("Test mode enabled - limiting requirements")
            sifp_logger.info(f"Limited to {len(selected_requirements)} requirements for testing")
            print(f"TEST MODE: Limited requirements")
            print(f"   Original: {original_count} → Limited: {len(selected_requirements)}")
        
        if not selected_requirements:
            sifp_logger.warning(f"No {requirement_type} requirements found to process")
            print(f"WARNING: No {requirement_type} requirements found to process")
            return []
            
        sifp_logger.debug(f"Processing {len(selected_requirements)} {requirement_type} requirements")
        print(f"\nProcessing {len(selected_requirements)} {requirement_type.upper()} requirements...")
        print("-" * 60)
        
        # Process requirements with detailed logging
        try:
            sifp_logger.debug("Starting SiFP batch processing")
            estimations = await workflow.process_requirements_batch(selected_requirements)
            
            if estimations:
                sifp_logger.info(f"Successfully generated {len(estimations)} SiFP estimations")
                print(f"Successfully generated {len(estimations)} SiFP estimations")
                
                # Log estimation details
                for est in estimations:
                    req_id = est.get('requirement_id', 'Unknown')
                    sifp_logger.debug(f"SiFP estimation details for {req_id}: {est}")
            else:
                sifp_logger.warning("No SiFP estimations were generated")
                print(f"WARNING: No SiFP estimations were generated")
                
            return estimations
            
        except Exception as e:
            sifp_logger.error(f"SiFP batch processing error: {str(e)}", exc_info=True)
            print(f"ERROR: SiFP batch processing error: {str(e)}")
            raise
        
    except Exception as e:
        sifp_logger.error(f"SiFP requirements processing error: {str(e)}", exc_info=True)
        print(f"ERROR: SiFP requirements processing error: {str(e)}")
        raise

print("SiFP helper functions defined successfully!")

In [ ]:
# Cell [4] - Execute SiFP Requirements Processing
# Purpose: Run the main SiFP processing workflow and display detailed results
# Dependencies: process_sifp_requirements function from Cell 3, workflow from Cell 2
# Breadcrumbs: Helper Functions -> Main Processing -> Results Display

def display_sifp_results(results):
    """Display detailed SiFP estimation results"""
    print(f"\nSiFP Estimation Results Analysis")
    print("=" * 80)
    print(f"Total estimations processed: {len(results)}")
    
    if not results:
        print("WARNING: No estimations to display")
        return
    
    print(f"\nDetailed SiFP Results:")
    print("-" * 80)
    
    for i, estimation in enumerate(results, 1):
        try:
            meta_judgment = estimation.get('meta_judgment', {})
            requirement_id = meta_judgment.get('requirement_id', 'N/A')
            
            print(f"\nEstimation {i} - Requirement ID: {requirement_id}")
            print("=" * 60)
            
            # Judge evaluation
            print(f"Judge Evaluation:")
            print(f"   Score: {meta_judgment.get('final_score', 'N/A')}")
            print(f"   Confidence: {meta_judgment.get('confidence', 'N/A')}")
            print(f"   Is Valid: {'YES' if meta_judgment.get('is_valid', False) else 'NO'}")
            
            # UGEP details
            final_estimation = meta_judgment.get('final_estimation', {})
            ugeps = final_estimation.get('ugeps', [])
            print(f"\nUGEPs (User Generic Elementary Processes):")
            if ugeps:
                for ugep in ugeps:
                    description = ugep.get('description', ugep.get('name', 'No description'))
                    weight = ugep.get('weight', 'N/A')
                    type_ = ugep.get('type', 'N/A')
                    print(f"   • {description} (Weight: {weight}, Type: {type_})")
            else:
                print(f"   No UGEPs identified")
            
            # UGDG details
            ugdgs = final_estimation.get('ugdgs', [])
            print(f"\nUGDGs (User Generic Data Groups):")
            if ugdgs:
                for ugdg in ugdgs:
                    description = ugdg.get('description', ugdg.get('name', 'No description'))
                    weight = ugdg.get('weight', 'N/A')
                    type_ = ugdg.get('type', 'N/A')
                    print(f"   • {description} (Weight: {weight}, Type: {type_})")
            else:
                print(f"   No UGDGs identified")
            
            # SiFP points breakdown
            sifp_points = final_estimation.get('sifp_points', {})
            print(f"\nSiFP Points Breakdown:")
            print(f"   ADD (Additions): {sifp_points.get('add', 0)}")
            print(f"   CHG (Changes): {sifp_points.get('chg', 0)}")
            print(f"   DEL (Deletions): {sifp_points.get('del', 0)}")
            print(f"   AUX (Auxiliary): {sifp_points.get('aux', 0)}")
            print(f"   TOTAL: {sifp_points.get('total', 0)} SiFP")
            
            # Reasoning
            reasoning = meta_judgment.get('reasoning', 'No reasoning provided')
            print(f"\nReasoning:")
            print(f"   {reasoning[:300]}{'...' if len(reasoning) > 300 else ''}")
            
            print("-" * 60)
            
        except Exception as e:
            sifp_logger.error(f"Error displaying estimation {i}: {str(e)}", exc_info=True)
            print(f"ERROR displaying estimation {i}: {str(e)}")
            continue
    
    # Summary statistics
    total_sifp = sum(
        est.get('meta_judgment', {}).get('final_estimation', {}).get('sifp_points', {}).get('total', 0)
        for est in results
    )
    valid_estimations = sum(
        1 for est in results 
        if est.get('meta_judgment', {}).get('is_valid', False)
    )
    
    print(f"\nSummary Statistics:")
    print(f"   Total Estimations: {len(results)}")
    print(f"   Valid Estimations: {valid_estimations} ({valid_estimations/len(results)*100:.1f}%)")
    print(f"   Total SiFP Points: {total_sifp}")
    print(f"   Average SiFP per Requirement: {total_sifp/len(results):.2f}")

# Configure logging for processing
sifp_logger = setup_logger("src.requirements_analyzer.sifp_workflow", logging.DEBUG)

# Execute the main SiFP processing
print(f"\nStarting SiFP Processing for {CONFIG['REQUIREMENT_TYPE'].upper()} Requirements")
print("=" * 80)

try:
    results = await process_sifp_requirements()
    
    sifp_logger.debug(f"Processing completed. Got {len(results)} results")
    sifp_logger.debug(f"Full results: {results}")
    
    # Display detailed results
    display_sifp_results(results)
    
    print(f"\nSiFP processing completed successfully!")
    
except Exception as e:
    print(f"Error during SiFP processing: {str(e)}")
    sifp_logger.error("SiFP processing failed", exc_info=True)
    raise

In [ ]:
# Cell [5] - Store SiFP Results in Neo4j and Files
# Purpose: Store the processed SiFP estimations in Neo4j database and save to JSON files
# Dependencies: results from Cell 4, workflow and neo4j_client from Cell 2, CONFIG from Cell 0
# Breadcrumbs: Results Processing -> Data Storage -> Neo4j & File Persistence

async def store_sifp_results(results: List[dict], 
                          workflow: SIFPWorkflow,
                          neo4j_client: Neo4jClient):
    """
    Store SiFP estimation results in Neo4j and JSON files.
    
    Args:
        results: List of estimation results from the workflow
        workflow: Initialized SiFP workflow
        neo4j_client: Neo4j client instance
    """
    storage_logger = setup_logger("notebooks.sifp_estimation", logging.DEBUG)
    
    if not results:
        print("WARNING: No results to store")
        return
    
    print(f"\nStoring SiFP Results")
    print("=" * 80)
    print(f"Model: {CONFIG['MODEL_NAME']}")
    print(f"Project: {CONFIG['NEO4J_PROJECT_NAME']}")
    print(f"Requirement Type: {CONFIG['REQUIREMENT_TYPE'].upper()}")
    print(f"Results to store: {len(results)}")
    
    try:
        # Prepare results directory with requirement type
        timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
        model_name = CONFIG['MODEL_NAME'].replace(".", "_")
        results_dir = pathlib.Path(f"results/sifp_estimations/{CONFIG['REQUIREMENT_TYPE']}") / model_name / timestamp
        results_dir.mkdir(parents=True, exist_ok=True)
        
        storage_logger.info(f"Processing {len(results)} {CONFIG['REQUIREMENT_TYPE']} estimation results for project {CONFIG['NEO4J_PROJECT_NAME']}")
        
        # Store results in Neo4j and prepare JSON data
        all_results = []
        successful_stores = 0
        failed_stores = 0
        
        for i, estimation in enumerate(results, 1):
            requirement_id = estimation.get("requirement_id")
            if not requirement_id:
                storage_logger.warning(f"Missing requirement_id in estimation: {estimation}")
                failed_stores += 1
                continue
                
            try:
                # Store in Neo4j with project name parameter
                await neo4j_client.store_sifp_results(
                    requirement_id=requirement_id,
                    model_name=CONFIG['MODEL_NAME'],
                    project_name=CONFIG['NEO4J_PROJECT_NAME'],
                    estimation_results=estimation
                )
                
                # Add to JSON results with project name
                all_results.append({
                    "timestamp": timestamp,
                    "model": CONFIG['MODEL_NAME'],
                    "requirement_type": CONFIG['REQUIREMENT_TYPE'],
                    "requirement_id": requirement_id,
                    "project_name": CONFIG['NEO4J_PROJECT_NAME'],
                    **estimation
                })
                
                storage_logger.debug(f"Stored results for {CONFIG['REQUIREMENT_TYPE']} requirement {requirement_id} in project {CONFIG['NEO4J_PROJECT_NAME']}")
                successful_stores += 1
                
                # Progress indicator
                if i % 5 == 0 or i == len(results):
                    print(f"   Progress: {i}/{len(results)} estimations processed")
                
            except Exception as e:
                storage_logger.error(f"Failed to store results for {CONFIG['REQUIREMENT_TYPE']} requirement {requirement_id} in project {CONFIG['NEO4J_PROJECT_NAME']}: {str(e)}")
                failed_stores += 1
                continue
        
        # Save consolidated JSON file
        results_file = results_dir / f"sifp_{CONFIG['REQUIREMENT_TYPE']}_estimations.json"
        with open(results_file, "w") as f:
            json.dump({
                "metadata": {
                    "timestamp": timestamp,
                    "model": CONFIG['MODEL_NAME'],
                    "requirement_type": CONFIG['REQUIREMENT_TYPE'],
                    "project_name": CONFIG['NEO4J_PROJECT_NAME'],
                    "total_requirements": len(results),
                    "processed_requirements": len(all_results)
                },
                "estimations": all_results
            }, f, indent=2)
            
        storage_logger.info(f"Saved {CONFIG['REQUIREMENT_TYPE']} results to {results_file}")
        
        # Storage summary
        print(f"\nStorage Summary:")
        print(f"   Successfully stored: {successful_stores}")
        print(f"   Failed to store: {failed_stores}")
        print(f"   Success rate: {successful_stores/(successful_stores+failed_stores)*100:.1f}%")
        print(f"   Results file: {results_file}")
        
        if failed_stores > 0:
            print(f"WARNING: Some results failed to store. Check logs for details.")
        else:
            print(f"All results stored successfully!")
        
    except Exception as e:
        storage_logger.error(f"Error storing {CONFIG['REQUIREMENT_TYPE']} results: {str(e)}", exc_info=True)
        print(f"Error storing results: {str(e)}")
        raise

# Execute storage process using the results from Cell 4
try:
    await store_sifp_results(results, workflow, neo4j_client)
except Exception as e:
    print(f"Error during storage operation: {str(e)}")
    logging.getLogger().error("Failed to store SiFP results", exc_info=True)
    raise

In [ ]:
# Cell [6] - Cleanup Connections and Finalize SiFP Workflow
# Purpose: Properly close all database connections and finalize the SiFP workflow
# Dependencies: Clients from Cell 2
# Breadcrumbs: Processing Complete -> Connection Cleanup -> SiFP Finalization

async def cleanup_sifp_connections():
    """
    Properly close all database connections and clean up SiFP resources
    """
    cleanup_logger = logging.getLogger("sifp_cleanup")
    cleanup_errors = []
    
    print(f"\nCleaning Up SiFP Connections")
    print("=" * 80)
    
    # Close Neo4j connection
    try:
        await neo4j_client.close()
        cleanup_logger.info("Neo4j connection closed successfully")
        print("Neo4j connection closed successfully")
    except Exception as e:
        error_msg = f"Neo4j connection cleanup failed: {str(e)}"
        cleanup_logger.error(f"{error_msg}")
        print(f"Error closing Neo4j connection: {str(e)}")
        cleanup_errors.append(error_msg)

    # Close Redis connection
    try:
        await redis_client.client.aclose()
        cleanup_logger.info("Redis connection closed successfully")
        print("Redis connection closed successfully")
    except Exception as e:
        error_msg = f"Redis connection cleanup failed: {str(e)}"
        cleanup_logger.error(f"{error_msg}")
        print(f"Error closing Redis connection: {str(e)}")
        cleanup_errors.append(error_msg)
    
    # Final status
    if cleanup_errors:
        print(f"\nCleanup completed with {len(cleanup_errors)} errors")
        for error in cleanup_errors:
            print(f"   • {error}")
    else:
        print(f"\nAll connections cleaned up successfully")
    
    return len(cleanup_errors) == 0

# Execute cleanup
cleanup_success = await cleanup_sifp_connections()

# Final SiFP workflow summary
print(f"\nSiFP Function Point Estimation Complete!")
print("=" * 80)
print(f"Model Used: {CONFIG['MODEL_NAME']}")
print(f"Project: {CONFIG['NEO4J_PROJECT_NAME']}")
print(f"Requirement Type: {CONFIG['REQUIREMENT_TYPE'].upper()}")
print(f"Test Mode: {CONFIG['TEST_MODE']}")
print(f"Results Processed: {len(results) if 'results' in locals() else 'N/A'}")
print(f"Log File: {CONFIG['LOG_FILE']}")
print(f"Cleanup Status: {'Success' if cleanup_success else 'With Warnings'}")

# Calculate total SiFP points if results exist
if 'results' in locals() and results:
    total_sifp = sum(
        est.get('meta_judgment', {}).get('final_estimation', {}).get('sifp_points', {}).get('total', 0)
        for est in results
    )
    valid_estimations = sum(
        1 for est in results 
        if est.get('meta_judgment', {}).get('is_valid', False)
    )
    
    print(f"\nSiFP Summary:")
    print(f"   Total SiFP Points: {total_sifp}")
    print(f"   Valid Estimations: {valid_estimations}/{len(results)}")
    print(f"   Average SiFP per Requirement: {total_sifp/len(results):.2f}")

# Log final completion
completion_logger = logging.getLogger("sifp_completion")
completion_logger.info(f"SiFP estimation workflow completed successfully")
completion_logger.info(f"Model: {CONFIG['MODEL_NAME']}, Type: {CONFIG['REQUIREMENT_TYPE']}, Results: {len(results) if 'results' in locals() else 'N/A'}")

print(f"\nNext Steps:")
print(f"   • Check the log file for detailed processing information")
print(f"   • Review stored SiFP results in Neo4j database")
print(f"   • Analyze function point estimates for project planning")
print(f"   • Compare results across different models if needed")
print(f"\nSiFP workflow completed!")

In [ ]:
# Cell [7] - Display Final SiFP Statistics (Optional)
# Purpose: Show comprehensive statistics and analysis of SiFP estimation results
# Dependencies: results from Cell 4
# Breadcrumbs: Workflow Complete -> Final Analysis -> Statistics Display

if 'results' in locals() and results:
    print(f"\nComprehensive SiFP Analysis")
    print("=" * 80)
    
    # Extract all SiFP data for analysis
    sifp_data = []
    for est in results:
        meta_judgment = est.get('meta_judgment', {})
        final_estimation = meta_judgment.get('final_estimation', {})
        sifp_points = final_estimation.get('sifp_points', {})
        
        sifp_data.append({
            'requirement_id': meta_judgment.get('requirement_id', 'Unknown'),
            'is_valid': meta_judgment.get('is_valid', False),
            'confidence': meta_judgment.get('confidence', 0),
            'total_sifp': sifp_points.get('total', 0),
            'add_points': sifp_points.get('add', 0),
            'chg_points': sifp_points.get('chg', 0),
            'del_points': sifp_points.get('del', 0),
            'aux_points': sifp_points.get('aux', 0),
            'ugep_count': len(final_estimation.get('ugeps', [])),
            'ugdg_count': len(final_estimation.get('ugdgs', []))
        })
    
    # Calculate comprehensive statistics
    valid_data = [d for d in sifp_data if d['is_valid']]
    
    if valid_data:
        total_sifp = sum(d['total_sifp'] for d in valid_data)
        avg_sifp = total_sifp / len(valid_data)
        max_sifp = max(d['total_sifp'] for d in valid_data)
        min_sifp = min(d['total_sifp'] for d in valid_data)
        
        print(f"SiFP Point Distribution:")
        print(f"   Total SiFP Points: {total_sifp}")
        print(f"   Average per Requirement: {avg_sifp:.2f}")
        print(f"   Maximum: {max_sifp}")
        print(f"   Minimum: {min_sifp}")
        
        # Breakdown by operation type
        total_add = sum(d['add_points'] for d in valid_data)
        total_chg = sum(d['chg_points'] for d in valid_data)
        total_del = sum(d['del_points'] for d in valid_data)
        total_aux = sum(d['aux_points'] for d in valid_data)
        
        print(f"\nOperation Type Breakdown:")
        print(f"   ADD (Additions): {total_add} ({total_add/total_sifp*100:.1f}%)")
        print(f"   CHG (Changes): {total_chg} ({total_chg/total_sifp*100:.1f}%)")
        print(f"   DEL (Deletions): {total_del} ({total_del/total_sifp*100:.1f}%)")
        print(f"   AUX (Auxiliary): {total_aux} ({total_aux/total_sifp*100:.1f}%)")
        
        # Component analysis
        total_ugeps = sum(d['ugep_count'] for d in valid_data)
        total_ugdgs = sum(d['ugdg_count'] for d in valid_data)
        
        print(f"\nComponent Analysis:")
        print(f"   Total UGEPs: {total_ugeps}")
        print(f"   Total UGDGs: {total_ugdgs}")
        print(f"   Average UGEPs per Requirement: {total_ugeps/len(valid_data):.2f}")
        print(f"   Average UGDGs per Requirement: {total_ugdgs/len(valid_data):.2f}")
        
        # Quality metrics
        avg_confidence = sum(d['confidence'] for d in valid_data) / len(valid_data)
        
        print(f"\nQuality Metrics:")
        print(f"   Valid Estimations: {len(valid_data)}/{len(results)} ({len(valid_data)/len(results)*100:.1f}%)")
        print(f"   Average Confidence: {avg_confidence:.2f}")
        
        # Top requirements by SiFP points
        top_requirements = sorted(valid_data, key=lambda x: x['total_sifp'], reverse=True)[:3]
        
        print(f"\nTop Requirements by SiFP Points:")
        for i, req in enumerate(top_requirements, 1):
            print(f"   {i}. {req['requirement_id']}: {req['total_sifp']} SiFP")
    
    else:
        print("WARNING: No valid estimations found for analysis")

else:
    print("WARNING: No results available for analysis")

print(f"\nSiFP Estimation Workflow Summary Complete!")
print("=" * 80)